In [9]:
import pandas as pd
import geoviews as gv
import geoviews.feature as gf
from geopy.geocoders import Nominatim
import hvplot.pandas  # Enables hvPlot for DataFrames
from bokeh.plotting import show

# Enable Bokeh extension for GeoViews
gv.extension('bokeh')




In [14]:
# Provide the correct path to the CSV file
file_path = 'Output_data/cities.csv'

# Load the data into the DataFrame
city_data_df = pd.read_csv(file_path)

city_data_df.head()



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


In [18]:
# Create a scatter plot humidity heatmap
humidity_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    cmap='viridis',   
    tiles='OSM',
    width=700,
    height=500,
    title='Humidity Heatmap',
)
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (Humidity)

In [20]:
# Filter cities based on ideal weather conditions
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

ideal_cities_df.head()
print(f"Number of ideal cities: {len(ideal_cities_df)}")
print(ideal_cities_df)

Number of ideal cities: 17
     City_ID            City      Lat       Lng  Max Temp  Humidity  \
45        45           kapaa  22.0752 -159.3190     22.99        84   
51        51            hilo  19.7297 -155.0900     26.27        83   
63        63           banda  25.4833   80.3333     24.62        52   
81        81   makakilo city  21.3469 -158.0858     21.66        81   
152      152         kahului  20.8947 -156.4700     23.80        60   
197      197             gat  31.6100   34.7642     24.38       100   
211      211          laguna  38.4210 -121.4238     21.67        79   
240      240     tikaitnagar  26.9500   81.5833     23.56        59   
265      265     san quintin  30.4833 -115.9500     21.20        74   
340      340   santa rosalia  27.3167 -112.2833     24.62        56   
363      363          narwar  25.6500   77.9000     22.35        55   
375      375    port hedland -20.3167  118.5667     21.03        73   
381      381       roebourne -20.7833  117.1333   

In [21]:
# Create a DataFrame to store hotel names along with city information
hotel_df = ideal_cities_df[['City', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df['Country'] = ""
hotel_df.head()


,City,Lat,Lng,Humidity,Hotel Name,Country
45,kapaa,22.0752,-159.3190,84,,
51,hilo,19.7297,-155.0900,83,,
63,banda,25.4833,80.3333,52,,
81,makakilo city,21.3469,-158.0858,81,,
152,kahului,20.8947,-156.4700,60,,


In [23]:

url = "https://api.geoapify.com/v1/api/geocode/reverse?lat=40.763843&lon=-73.972969&apiKey=api_keys.py"
geoapify_key = "api_keys.py"  


# Loop through each city to find a nearby hotel
for index, row in hotel_df.iterrows():
    lat, lng = row["Lat"], row["Lng"]

    try:
        # Make the API request to find hotels within 10,000 meters
        url = f"https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:{lng},{lat},10000&limit=1&apiKey={geoapify_key}"
        response = requests.get(url).json()

        # Extract the first hotel name and country if available
        if response["features"]:
            hotel_name = response["features"][0]["properties"]["name"]
            country = response["features"][0]["properties"]["country"]
        else:
            hotel_name, country = "No Hotel Found", "Unknown"

        # Update the DataFrame
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        hotel_df.loc[index, "Country"] = country

    except Exception as e:
        print(f"Error fetching hotel for {row['City']}: {e}")

hotel_df.head()


Error fetching hotel for kapaa: name 'requests' is not defined
Error fetching hotel for hilo: name 'requests' is not defined
Error fetching hotel for banda: name 'requests' is not defined
Error fetching hotel for makakilo city: name 'requests' is not defined
Error fetching hotel for kahului: name 'requests' is not defined
Error fetching hotel for gat: name 'requests' is not defined
Error fetching hotel for laguna: name 'requests' is not defined
Error fetching hotel for tikaitnagar: name 'requests' is not defined
Error fetching hotel for san quintin: name 'requests' is not defined
Error fetching hotel for santa rosalia: name 'requests' is not defined
Error fetching hotel for narwar: name 'requests' is not defined
Error fetching hotel for port hedland: name 'requests' is not defined
Error fetching hotel for roebourne: name 'requests' is not defined
Error fetching hotel for saint-francois: name 'requests' is not defined
Error fetching hotel for capoterra: name 'requests' is not defined
Er

,City,Lat,Lng,Humidity,Hotel Name,Country
45,kapaa,22.0752,-159.3190,84,,
51,hilo,19.7297,-155.0900,83,,
63,banda,25.4833,80.3333,52,,
81,makakilo city,21.3469,-158.0858,81,,
152,kahului,20.8947,-156.4700,60,,


In [25]:
# Create a map showing hotels for ideal vacation cities
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    size=10, 
    color='blue', 
    tiles='OSM', 
    hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
    frame_width=800, 
    frame_height=500, 
    title='Hotels Near Ideal Vacation Cities'
)

# Display the map
hotel_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [x,y]   (City,Country,Hotel Name,Humidity)